# 📓 ACCESS-ESM15-write-zarr

**Author:** Thomas Moore  
**Date:** 2025-10-31 <br>
**Updated:** 2025-11-13
**Environment:** `pangeo_csepta` running on `Gadi` ARE  
**Tags:** sandbox, ARD, ACCESS-ESM1.5

---

### 📘 Description

This notebook attempts to bring together all the disperate workplows for this task over the year(s).  Issue: [https://github.com/Thomas-Moore-Creative/Pacific-Tuna-Climate-Response/issues/51](https://github.com/Thomas-Moore-Creative/Pacific-Tuna-Climate-Response/issues/51) and sub-issues

In [1]:
Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

# Software

### ACDtools

In [2]:
!pip install --user -e /g/data/es60/users/thomas_moore/code/ACDtools

Obtaining file:///g/data/es60/users/thomas_moore/code/ACDtools
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ACDtools (pyproject.toml) ... done
  Created wheel for ACDtools: filename=acdtools-0.1-0.editable-py3-none-any.whl size=3553 sha256=6d2df8821af4295407434d9c961fcadf1f7d162f23a22316ff6a7c9ba247f737
  Stored in directory: /scratch/es60/thomas_moore/tmp/pip-ephem-wheel-cache-8mq84xhk/wheels/b6/a3/f2/6ce45fbdc116ad50e421d6a11cb060cc796e867501807af446
Successfully built ACDtools
  Attempting uninstall: ACDtools
    Found existing installation: ACDtools 0.1
    Uninstalling ACDtools-0.1:
      Successfully uninstalled ACDtools-0.1


In [3]:
# Enable autoreload in the notebook
%load_ext autoreload
%autoreload 1 
%aimport ACDtools.util
%aimport ACDtools.ard
%aimport ACDtools.ocean
%aimport ACDtools.plot
# Importing from your local package util.py
from ACDtools import util
from ACDtools import ard
from ACDtools import ocean
from ACDtools import plot

In [4]:
import datetime
from pprint import pprint
import intake
import sys
from contextlib import redirect_stdout
import math

## Cluster

In [5]:
client, cluster = util.start_dask_cluster_from_config('netcdf_work')

Cluster started with 32 workers.
Dashboard available at: /proxy/8787/status


## Functions

In [6]:
# develop any new functions here

# settings

In [8]:
job_config_path = '/g/data/es60/users/thomas_moore/code/ACDtools/job_config.yaml'
job_config_dict = ACDtools.util.load_config(job_config_path)
job_config_dict

{'paths': {'write_dir': '/scratch/es60/ard/models/ACCESS-ESM15/ARD/',
  'save_coords_dir': '/scratch/es60/ard/models/ACCESS-ESM15/ARD/coords/',
  'log_dir': '/scratch/es60/ard/models/ACCESS-ESM15/ARD/logs/'},
 'dask_cluster': {'netcdf_work': {'n_workers': 'None',
   'threads_per_worker': 1,
   'memory_limit': 'None'},
  'zarr_work': {'n_workers': 'None',
   'threads_per_worker': 'None',
   'memory_limit': 'None'}},
 'catalog_search_query_dict': {'experiment_id': 'ssp585',
  'source_id': 'ACCESS-ESM1-5',
  'variable_id': 'uo',
  'table_id': 'Omon',
  'version': 'latest'},
 'time_trim': {'enabled': True, 'start': None, 'end': '2100-12-31'},
 'chunking_key': 'ACCESS_ESM15_3D',
 'chunking': {'ACCESS_ESM15_2D': {'chunks': {'member': 1,
    'time': 220,
    'i': 360,
    'j': 300}},
  'ACCESS_ESM15_3D': {'chunks': {'member': 1,
    'time': 12,
    'lev': -1,
    'i': -1,
    'j': -1}}}}

# workflow with ACCESS-NRI catalog that uses NCI catalog

In [9]:
catalog = intake.cat.access_nri

In [10]:
%%time
# --- build log filename dynamically ---
logfile = f"{job_config_dict['paths']['log_dir']}log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

with open(logfile, "w") as f, redirect_stdout(f):
    #####
    print('Running: ' +
          job_config_dict['catalog_search_query_dict']['source_id'] +
          ' ' + job_config_dict['catalog_search_query_dict']['experiment_id'] +
          ' ' + job_config_dict['catalog_search_query_dict']['variable_id'] +
          ' ' + job_config_dict['catalog_search_query_dict']['table_id'] +
          ' ' + job_config_dict['catalog_search_query_dict']['version']
         )
    pprint(job_config_dict)
    # load catalog
    cmip6_fs38_datastore = ACDtools.util.load_cmip6_fs38_datastore()
    # search catalog for list of files
    search = cmip6_fs38_datastore.search(**job_config_dict['catalog_search_query_dict'])
    # load into one object using xarray kwags for chunking and handling cftime
    ####
    #ds = load_ACCESS_ESM(search,use_cftime=True,chunking_key=job_config_dict['chunking_key'])
    ####
    ds = ACDtools.ard.load_ACCESS_ESM_ensemble(search,use_cftime=True,chunking_key=job_config_dict['chunking_key'])
    # save and drop multidimensional coordinates
    ds = ACDtools.ard.save_n_drop_multidim_lat_lon(ds,save_coords_dir=job_config_dict['paths']['save_coords_dir'],
                                      variable_name = job_config_dict['catalog_search_query_dict']['variable_id'])
    
    # option to trim time axis
    trim = job_config_dict['time_trim']
    print(trim)
    if trim.get("enabled", True):
        start = trim.get("start") or None
        end = trim.get("end") or None
        ds = ds.sel(time=slice(start, end))
    # remove encoding
    ACDtools.util.remove_encoding(ds)
    # write out zarr
    current_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    print('Started zarr write at '+current_datetime)
    filename = (
    f"{job_config_dict['paths']['write_dir']}"
    + f"{job_config_dict['catalog_search_query_dict']['source_id']}."
    + f"{job_config_dict['catalog_search_query_dict']['experiment_id']}."
    + f"{job_config_dict['catalog_search_query_dict']['variable_id']}."
    + f"base.v{current_datetime}.zarr"
    )
    print('Zarr filename = '+filename )
    ds.to_zarr(filename,consolidated=True)
    end_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    print('Finished at '+ end_datetime)
    ##### start log cluster settings
    # Get existing client (do NOT create a new one)
    client = client.current()   # or Client() if not yet connected

    info = client.scheduler_info()["workers"]

    # total cores (threads)
    n_workers = len(info)
    total_threads = sum(w.get("nthreads", 0) for w in info.values())

    # helper: MiB/GB conversion using powers of 1024
    def to_gib(bytes_):
        return bytes_ / (1024 ** 3)

    # 1) Try configured memory_limit
    limits = {addr: w.get("memory_limit", 0) for addr, w in info.items()}

    # 2) For any worker reporting 0/None (unlimited), fetch actual host RAM via psutil on the worker
    need_actual = [addr for addr, lim in limits.items() if not lim or lim == 0 or math.isclose(lim, 0.0)]
    
    if need_actual:
        # run a tiny function on workers to read psutil.virtual_memory().total
        def _get_total_ram():
            try:
                import psutil
                return psutil.virtual_memory().total
            except Exception:
                # fallback: dask.system.MEMORY_LIMIT may help on some installs
                try:
                    from dask.system import MEMORY_LIMIT
                    return MEMORY_LIMIT or 0
                except Exception:
                    return 0
    
        actual = client.run(_get_total_ram)  # returns {worker_addr: bytes}
        # fill in missing/zero limits with actual system RAM
        for addr in need_actual:
            if addr in actual and actual[addr]:
                limits[addr] = actual[addr]
    
    total_mem_bytes = sum(limits.values())
    total_mem_gib = to_gib(total_mem_bytes)/n_workers
    
    # Build a short log block
    log_lines = [
        "Dask cluster resources:",
        f"  Workers:      {n_workers}",
        f"  Total cores:  {total_threads}",
        f"  Total memory: {total_mem_gib:.2f} GiB",
    ]
    
    print("\n".join(log_lines))
    ########### end log cluster settings
    print("==== Log complete ====")
# ---------------------------------------------------------
print(f"✅ Log written to {logfile}")


✅ Log written to /scratch/es60/ard/models/ACCESS-ESM15/ARD/logs/log_20251113_213306.txt
CPU times: user 7min 48s, sys: 1min 55s, total: 9min 44s
Wall time: 20min 41s


# $THE\ END$

# testing

In [6]:
import xarray as xr

In [13]:
thetao = xr.open_zarr('/scratch/es60/ard/models/ACCESS-ESM15/ARD/ACCESS-ESM1-5.historical.thetao.base.v20251106_153203.zarr',consolidated=True)

In [14]:
max_temp = thetao.thetao.max()

In [15]:
%%time
max_temp = max_temp.compute()

CPU times: user 1min 24s, sys: 12.7 s, total: 1min 37s
Wall time: 3min 23s


In [16]:
max_temp

<xarray.DataArray 'thetao' ()> Size: 4B
array(35.09579, dtype=float32)

In [11]:
chl

<xarray.Dataset> Size: 2TB
Dimensions:   (member: 40, time: 1980, lev: 50, j: 300, i: 360, bnds: 2)
Coordinates:
  * i         (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
  * j         (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * lev       (lev) float64 400B 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
    lev_bnds  (lev, bnds) float64 800B dask.array<chunksize=(50, 2), meta=np.ndarray>
  * member    (member) <U9 1kB 'r1i1p1f1' 'r2i1p1f1' ... 'r39i1p1f1' 'r40i1p1f1'
  * time      (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T1...
Dimensions without coordinates: bnds
Data variables:
    chl       (member, time, lev, j, i) float32 2TB dask.array<chunksize=(1, 12, 50, 300, 360), meta=np.ndarray>
Attributes: (12/61)
    Conventions:                      CF-1.7 CMIP-6.2
    NOTE on coordinates:              the multidimensional latitude and longi...
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            168012.0
    ...                               ...
    table_id:                         Omon
    table_info:                       Creation Date:(30 April 2019) MD5:5bd75...
    title:                            ACCESS-ESM1-5 output prepared for CMIP6
    variable_id:                      chl
    variant_label:                    r21i1p1f1
    version:                          v20200922